In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import StandardScaler


import optuna

import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\yashoda PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pip install optuna



  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/69/60/87a06ef66b34cbe2f2eb0ab66f003664404a7f40c21403a69fad7e28a82b/optuna-3.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/a2/8b/46919127496036c8e990b2b236454a0d8655fd46e1df2fd35610a9cbc842/alembic-1.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for cmaes>=0.10.0 from https://files.pythonhosted.org/packages/f7/46/7d9544d453346f6c0c405916c95fdb653491ea2e9976cabb810ba2fe8cd4/cmaes-0.10.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/78.7 kB ? eta -:--:--
     -------------------- ------------------- 41.0/78.7 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------- 78.7/78.7 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/404.2 kB ? eta -:--:--
   ----------------- ---------------------- 174.1/404.2 kB 5.3 MB/s eta 0:00:01
   -----

# Load the dataset and explore

In [5]:
df = pd.read_csv('parkinsons.csv')
df.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [6]:
# there are no null data
# number of entries is 195 (it is a tiny dataset)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [7]:
# heatmap shows a bunch of feature each other correlated each other:
# some analisys can be done here to understand which are really useful and which can be dropped
sns.heatmap(df.corr());

ValueError: could not convert string to float: 'phon_R01_S01_1'

In [ ]:
# this code can be enabled if you want to remove from datease the correlations computed above

#features_to_be_removed = set()

#for feature_tuple in highly_correlated_features:
#   features_to_be_removed.add(feature_tuple[1])

#df.drop(features_to_be_removed,inplace=True, axis=1)

In [9]:
# mutual info classification
from sklearn.feature_selection import mutual_info_classif

X = df.drop(['name','status'], axis=1)
y = df['status']

mutual_info = mutual_info_classif(X,y)
# print of mutual_info
from matplotlib.pyplot import figure

figure(figsize=(28, 6), dpi=80)
sns.barplot(X.columns, mutual_info);

TypeError: barplot() takes from 0 to 1 positional arguments but 2 were given

<Figure size 2240x480 with 0 Axes>

In [ ]:
# some feature shows outliers: more analysis and trials can be done here

for feature in df.columns:
    if feature == 'name' or feature == 'status':
        continue
    plt.figure()
    sns.boxplot(x="status", y=feature, data=df)

# First run with a classifier

In [ ]:
# prepare features and target
X = df.drop(['name','status'], axis=1)
y = df['status']

In [ ]:
# scale data
from sklearn.preprocessing import RobustScaler

robustScaler = RobustScaler()
robustScaler.fit(X)
robustScaler.transform(X)

In [ ]:
# split in train and test set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, shuffle=True, stratify=y, random_state=0)

In [ ]:
# use DecisionTree classifier
classifier = DecisionTreeClassifier(random_state=0)
classifier.fit(X_train,y_train)
print((classifier.score(X_train, y_train), classifier.score(X_test,y_test)))

In [ ]:
# plot confusion matrix for train set
plot_confusion_matrix(classifier, X_train, y_train, normalize='true', cmap='Blues',display_labels=["Healthy","Parkinson"]);

In [ ]:
# plot confusion matrix for test set
#
# result is quite good and confusion matrix is balanced
# if "Parkinson" is considered as "positive (P)", since we are talking about an illness
# it is nice that agorithm predicted more FP (predict "Parkinson" to "healthy")
# rather than TN ("Healthy" to "Parkinson")
plot_confusion_matrix(classifier, X_test, y_test,normalize='true', cmap='Blues',display_labels=["Healthy","Parkinson"]);

# Use Optuna to optimize hyperparameters

In [ ]:
def objective(trial):
    classifier_name = trial.suggest_categorical("classifier", ["DecisionTree"])
    c_criterion = trial.suggest_categorical("criterion", ["gini","entropy"])
    c_splitter = trial.suggest_categorical("splitter", ["best","random"])
    c_max_depth = trial.suggest_int("max_depth", 2, 32, log=True)
    c_min_samples_split = trial.suggest_int("min_samples_split", 2, 20, log=True)
    classifier_obj = DecisionTreeClassifier(criterion=c_criterion, splitter=c_splitter,
        max_depth=c_max_depth, min_samples_split = c_min_samples_split, random_state=0)

    score = cross_val_score(classifier_obj, X, y, n_jobs=-1, cv=10)
    accuracy = score.mean()
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)
    print(study.best_trial)

In [ ]:
# plot best params found
study.best_params

# Try again with hyperparameters found with Optuna

In [ ]:
# classifier with best parameters from optuna
classifier = DecisionTreeClassifier(
    criterion=study.best_params['criterion'],
    splitter=study.best_params['splitter'],
    max_depth=study.best_params['max_depth'],
    min_samples_split=study.best_params['min_samples_split'],
    random_state=0)
classifier.fit(X_train,y_train)
print((classifier.score(X_train, y_train), classifier.score(X_test,y_test)))

In [ ]:
# this is the best result out of Optuna's study on my pc
# (skip this to plot confusion matrix with parameter found)
classifier = DecisionTreeClassifier(
    criterion='entropy',
    splitter='best',
    max_depth=28,
    min_samples_split=2,
    random_state=0)
classifier.fit(X_train,y_train)
print((classifier.score(X_train, y_train), classifier.score(X_test,y_test)))

In [ ]:
# plot confusion matrix for train set again
plot_confusion_matrix(classifier, X_train,y_train, normalize='true', cmap='Blues',display_labels=["Healthy","Parkinson"]);

In [ ]:
# plot confusion matrix for test set again
# results are slightly better than the original run
plot_confusion_matrix(classifier, X_test, y_test,normalize='true', cmap='Blues',display_labels=["Healthy","Parkinson"]);